In [205]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import a_funciones as funciones  ###archivo de funciones propias
import numpy as np
import plotly.express as px
from sklearn.model_selection import RandomizedSearchCV
import joblib  ### para guardar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
import openpyxl
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier


In [206]:
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/df_final.csv", sep=",")
df2 = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/base16.csv", sep=",")

In [207]:
df.head(10)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsSinceLastPromotion,JobInvolvement,PerformanceRating,Attrition,retirementType,resignationReason
0,3,4,2,51,Travel_Rarely,Sales,6,2,Life Sciences,Female,...,11,0,1,6,0,3,3,Si,no resignation,no resignation
1,3,2,4,31,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,...,23,1,6,3,1,2,4,No,Resignation,Others
2,2,2,1,32,Travel_Frequently,Research & Development,17,4,Other,Male,...,15,3,5,2,0,3,3,Si,no resignation,no resignation
3,4,4,3,38,Non-Travel,Research & Development,2,5,Life Sciences,Male,...,11,3,13,5,7,2,3,Si,no resignation,no resignation
4,4,1,3,32,Travel_Rarely,Research & Development,10,1,Medical,Male,...,12,2,9,2,0,3,3,Si,no resignation,no resignation
5,3,2,2,46,Travel_Rarely,Research & Development,8,3,Life Sciences,Female,...,13,0,28,5,7,3,3,Si,no resignation,no resignation
6,1,3,1,28,Travel_Rarely,Research & Development,11,2,Medical,Male,...,20,1,5,2,0,3,4,No,Resignation,Stress
7,1,2,3,29,Travel_Rarely,Research & Development,18,3,Life Sciences,Male,...,22,3,10,2,0,3,4,Si,no resignation,no resignation
8,2,4,3,31,Travel_Rarely,Research & Development,1,3,Life Sciences,Male,...,21,0,10,2,7,3,4,Si,no resignation,no resignation
9,2,1,3,25,Non-Travel,Research & Development,7,4,Medical,Female,...,13,1,6,2,1,3,3,Si,no resignation,no resignation


## **Obtener dummies**

In [208]:
df1 = pd.get_dummies(df.drop('Attrition', axis=1))

# Agregar 'Attrition' de nuevo al DataFrame
df1['Attrition'] = df['Attrition']

In [209]:
### conversion a entero 
df1['Attrition'] = df1['Attrition'].map({'No': 0, 'Si': 1}).astype(int)


## **Seleccion de variables**

In [210]:
#####Separamos nuestra variable de las demas variables
X = df1.drop('Attrition',axis=1)
y = df1['Attrition']

In [211]:
######Seleccion de modelos
### 1. Regresion logistica 
m_rl = LogisticRegression(max_iter=1000, solver='liblinear', C=1.0)
### 2. Random forest
m_rf = RandomForestClassifier()
### 3 Arboles de decision
m_add = DecisionTreeClassifier()


In [212]:
modelos = list([m_rl, m_rf, m_add])

In [213]:
var_names=funciones.sel_variables(modelos,X,y,threshold="2.5*mean")
var_names.shape


(6,)

In [215]:
X2=X[var_names] ### matriz con variables seleccionadas
X2.info()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   Age                               4298 non-null   int64
 1   TotalWorkingYears                 4298 non-null   int64
 2   resignationReason_Others          4298 non-null   uint8
 3   resignationReason_no resignation  4298 non-null   uint8
 4   retirementType_Resignation        4298 non-null   uint8
 5   retirementType_no resignation     4298 non-null   uint8
dtypes: int64(2), uint8(4)
memory usage: 84.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   EnvironmentSatisfaction            4298 non-null   int64
 1   JobSatisfaction                    4298 non-null   int64
 2

In [ ]:
rmse_df=funciones.medir_modelos(modelos,"neg_root_mean_squared_error",X,y,4) ## base con todas las variables 
rmse_varsel=funciones.medir_modelos(modelos,"neg_root_mean_squared_error",X2,y,4) ### base con variables seleccionadas